# Loan calculator

Version 1.0

## Imports

In [ ]:
import numpy as np

## Functions

In [2]:
def PMT(Pv:int =1000, rate:float=1.0, per:int=1, fper:int=1):
    """ Calculates the payment for a loan based on constant payments and a constant interest rate.
    Args:
        - Pv : Present Value (starting value of the loan)
        - r : Annual Percent Rate (APR)
        - fn : fraction of period of interest periods per year
        - n : number of years
    Returns:
        float: PMT value
    """
    nper = per * fper # number of periodic payment
    pir = rate / fper # Periodic Interest Rate
    PMT = (Pv * pir) / (1 - (1 + pir)**(-nper))

    return -PMT



def IPMT(c:int, r:float, per:int, fper:int, pv:int):    
    """Calculate IPMT
    Args:
        - c is PMT()
        - r is Annual Rate
        - per is Payment Number
        - pv is Present Value
    Returns:
        float: IPMT value
    """
     
    pir = r / fper # Periodic Interest Rate
    
    
    IPMT = -(c * ((1 + pir)**(per-1) - 1) / pir + pv * (1 + pir)**(per-1)) * pir

    return IPMT


def PPMT(c:int, r:float, per:int, fper:int, pv:int):
    """Calculate PPMT
    Args:
        - c is PMT()
        - r is Rate
        - per is Payment Number
        - pv is Present Value
    Returns:
        float: PPMT value
    """
    
    PIR = r / fper # Periodic Interest Rate
    
    ppmt = c+(c * ((1 + PIR)**(per-1) - 1) / PIR + pv * (1 + PIR)**(per-1)) * PIR

    return ppmt

## Global Variables

In [5]:
Pv1 = 100000
r1 = 0.0300
fper1 = 12
per1 = 25
n1 = 300

In [4]:
pmt = PMT(Pv=Pv1, rate=r1, per=per1, fper=fper1 ) 
ipmt = IPMT(c=pmt, r=r1, per=n1, fper=fper1, pv=Pv1) 
ppmt = PPMT(c=pmt, r=r1, per=n1, fper=fper1, pv=Pv1) 

print( pmt )
print( ipmt)
print( ppmt )

-1430.4480079117482
-3.6265046541130963
-1426.821503257635
